In [2]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math
import json
import glob
import xlrd
from utilities import perform_stats, calc_z_scores,\
    get_enrichr_results, plot_results,\
    plot_results_in_singleplot, plot_results_in_singleplot_iteractive_legend,\
    count_words_in_df, plot_count_results, split_and_count_words

# Defining some import and export locations
location = 'rjsietsma'
read_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/'
data_expor_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Past_initial_data/'
img_output_dir = '/home/'+location+'/PycharmProjects/dsls_master_thesis/side_scripts/output_img/'

In [3]:
# Some bokeh stuff

from bokeh.plotting import figure, ColumnDataSource, output_file, show, save
from bokeh.models import HoverTool, WheelZoomTool, PanTool,\
    BoxZoomTool, ResetTool, TapTool, SaveTool, FactorRange
from bokeh.palettes import inferno, Spectral5
from bokeh.transform import factor_cmap

Sprint 28-04-2020 - 19-05-2020 (?)
=====================

- What I want to do:
  1. [Focus on fixing missing CAPICE precomputed scores.](#Fixing-CAPICE-precomputed-scores) __Done__
      - _Note: Find out how the script of Shuang works on a smaller scale, get an example where a CAPICE score is missing._
  2. [Normalize BM_ratio](#Normalize-BM_ratio) for gene length, so $\frac{n_{malignant}}{n_{benign}}$ per 1000 bases in a gene, instead of [$\frac{n_{malign}}{n_{benign}} \times n_{total}$](output_img/bm_ratio.png "BM_ratio").
  3. _(questionable)_ Fix PCA by identifying categorical features _(Questionable because should I still put time and focus in on the input data?)_
      - _Ask Shuang, Joeri and Krista in the PRU 28-04-2020 @ 13:00-14:00_
  4. [See what is required](#Required-for-data-sources) for the [data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) to be implemented into the input data.
      - _Note: as this is research, I can use sources that only have precomputed scores available, but just to keep in mind that I should mention that in the discussion._

- Notes:
  - If I find myself unfocussed:
      - Win a game of [solitaire](https://play.google.com/store/apps/details?id=com.lemongame.klondike.solitaire&hl=nl) (game is rigged, not every match is win-able).
      - Clean out my closet (it's messy, maybe to clean up the mess inside my head too).
      - Go for a walk around the neighbourhood.

<br/><br/>
<br/><br/>
### Fixing CAPICE precomputed scores

Problem: CAPICE precomputed file of all single nucleotide level variants contains caps.

Solution: Fix the underlying problem that causes these gaps.

Required: 
- Script to make and batch the CAPICE computing __(positive check)__
- Access to GCC cluster __(positive check)__
- Knowledge of job scheduler on GCC cluster __(positive check)__
- Smaller batches of SNV files to run locally  __(positive check)__

Possible cause:
- Incorrect batching on tabix.

31-04-2020 Problem:
- Got it working on Gearshift, but with a problem:
    - AttributeError: Can't get attribute 'DeprecationDict' on <module 'sklearn.utils.deprecation' from '/apps/software/PythonPlus/3.7.4-foss-2018b-v20.02.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py'>
- Which let's me to beleive that the model was trained in a previous version of sklearn.

<br/><br/>

- Update 10:51 : Model compatibility issue. Might have to retrain CAPICE on Gearshift using the latest xgboost, sklearn, scikit-learn, numpy and pandas just to be safe.

- Update 12:01 : Managed to get it working using python3.6 on calculon, essentially downgrading my own script (I only used python3.7 for F-strings)

- Update 15:55 : Found a very weird bug in the chunking. Seems like the script doesn't like to start at 0 with skiprows and 1000 nrows, solution: nrows - 2 when first iteration, then + 1, then start += nrows + 1...

- Update 04-05-2020 : Found out that the first iteration, an overhead of 2 variants was present. Adjusted the start with 2 to compensate. Seemed fine in the first 400000 CADD entries. (no duplicates nor missed entries). __Started full processing__

- Update 04-05-2020 : 
    - Add functionality to continue where left off if it crashes.
    - Make dummy script to check for duplicates or if the files are fully complete.
    
- Update 06-05-2020 : 
    - Functionaility to continue where left off is required. After 40 hours, it's still processing chromosome 1 after 7 million processed variants. 
        - Ideas: write json file with start and stop each iteration once that part is processed and added to the larger files. Should overwrite.
        - As for now, since this idea is not implemented yet, count the number of rows in the already processed files and set that as start.

- Update 11-05-2020:
    - The 4 days have ended, chromosome 1 has not been finished yet.
    - Output file of chromosome 1 has been renamed to a .tsv file and gzipped (saving about 5 gb of storage).
    - Adjustments to the original script have to be made.
   

#### Version 2.0

- Added gzip output by default
- Currently no function yet to continue where left off.
    - Ideas:
        - for line in gzip.open(file.gz): nlines+=1 (to trace back where was left off)
            - Do not know if this lead to any complications of duplicated entries. Saving the previous results in the RAM memory might solve this to compare to.
    - Then in the future:
        - Save a json of start and batch size each iteration to mark where the file was left of.

- Update 15-05-2020:
    - Program did not load in the previous results correctly, resulting in 12 duplicates (111400001-111400012) (including start and stop). Discuss with Joeri and Shuang what to do. It takes roughly 200 MB of ram to load in 2.1 million processed variants, calculating this for 111.4 million variants (that's how much there is in the file now), would come down to ~11GB of ram usage.

- Update 18-05-2020:
    - Script to remove duplicates from the CAPICE processed file has been run and duplicates have been removed.
    - First test runs have been ran, only for me to realised I messed up. I kept copying the main class, instead of the calculate_capice class. Final release should be on this date.
    

<br/><br/>
<br/><br/>
### Normalize BM_ratio

Problem: Larger genes have more benign and malignant samples, because they are larger and thus the chanses of variation on the reference read is higher.

Solution: Make a $n_{malignant}$ and $n_{benign}$ per $x$ bases in a gene, where {$x \in \mathbb{N}\ | x \geq 1 | x \leq 10000$}

<br/><br/>
<br/><br/>
### Required for data sources

Problem: Now that a lot of data sources have been found, annotation and merging has to be performed in order to use it as training data.
- _Additional problem: once a source has been proven beneficial, it has te be implemented to be annotated before capice fits and predicts a new sample._

Solution: Find out, feature by feature if genes that have a low AUC improve *__and that genes that have a high AUC do not perform worse__*

Required:
- Storage 
    - On GCC cluster for precomputed score files.
    - Locally IF file isn't that big or an API can be used.
- Data sources __([positive check](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit))__
- Script to annotate / merge


      

# CTEPG (Catching The Elusive Predictable Genes) 28-04-2020 - 04-05-2020
<br/><br/>

What have I done:

- [Enrichr Genes < 0.6](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=b7a959357214069d1905719220ed3e3b)
- [Enrichr Genes < 0.7](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7d252a3e1df275859228fd562e964344)
    - For reference: [Enrichr Genes < 0.85](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=60c74741c967c25331419bd2b5e7183e)
- [Finished investigating initial data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit)

<br/><br/>
<br/><br/>

What I'm planning to do this week:

- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores). __Done and not done, script has to have some additonal features__
    - If stuck, do not keep walking into the wall, continue below and contact Shuang and Joeri
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API). __Done__
    - Read into the [Enrichr API documentation](https://amp.pharm.mssm.edu/Enrichr/help#api)
- Mail Tsjerk that a meeting is apreciated. __Done, no response yet__
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p). __Done__
    - Select from enrichr all the genes involved in (lipid) metabolism and see if this explains all / most of the bad performing genes.
- Make model on bad performing genes and investigate further.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Fix PCA.
    - Find possible way to annotate genes for co-factors, sub-family etc.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 04-05-2020 - 11-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API).
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).
<br/><br/>

Planned for this week:
- Release precomputed CAPICE script v2.0
- Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-~0.75 and try to find features / links that would be beneficial to train a separate model on. (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - Find commonalities between y-axis 0-0.2 and x-axis 0-0.7
    - Make model on bad performing genes and investigate further.
        - Make plots combined of all sources, but color them by source.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 11-05-2020 - 18-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- Make more bokeh plots on enrichr results
    - ~[Investigate correlations between AUC's and Adjusted P-value](#Correlation-analysis-Enrichr-results).  (spearman / pearson, preverably spearman). Keep in mind that the distribution is not linear, but more like a normal distribution like.~
        - ~Make legend interactive to confirm normal distribution like.~
    - Chromosome X seems enriched for higher AUC thresholds. Bias in the source data? (Put in discussion). Lot's of developmental delay reseach is linked / correlated to chromosome X.

Planned for this week:
- ~Release precomputed CAPICE script v2.0.~
- ~Run correlation test for all possible sources on Enrichr to maximize chanses of finding stuff.~
    - ~Look into a good way to get all possible results from the enrichr API.~ __Not possible.__
- ~Escalate mail contact with Tsjerk to Martijn and Fenna.~
    - __Unneeded, Tsjerk replied.__
- ~Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-0.75 and try to find features / links that would be beneficial to train a separate model on.~ (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - ~Find commonalities between y-axis 0-0.2 and x-axis 0-0.7~
    - __Make model on bad performing genes and investigate further.__
        - ~Make plots combined of all sources, but color them by source.~
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - __Create outline for article.__
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
        - For first concept, it's due date is the last week of the resit exams on the 6th of July.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>
    
## For after I graduate:
<br/><br/>
Make a script that for every gene, for every gene makes a plot like [these](https://molgenis26.gcc.rug.nl/downloads/capice/plots/), but then interactive that if you hover over a point you see it's chromosomal location, ref and alt and maybe an RS number?

- If input of CAPICE is not CADD annotated, annotate in capice locally and only return the capice scores.
 







  
    

# Current Tickets:
<br/><br/>

## As student / Robert I want to map requirements for SNV and gene features.

- Content:

Overview of features has now been collected and processed (https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit). In order to use these features, knowledge has to be obtained for the requirements on said features including, but not limited to merging to the trianing dataset and storage.

- Acceptance criterea:
    - Does not take up terrabytes of storage.
        - _If it does, data sources might have to be merged 1 by 1, removing the source after the merge._
    - Can be merged without 3rd party software (tabix, python etc. allowed).
    - Data can easily be downloaded.

<br/><br/>

## As student / Robert I want to find commonalities between bad performing genes in CAPICE

- Content:

The enrichr tool has proven to be quite beneficial to understanding a set of bad performing genes. Further processing of these enrichr results is required in order to fully understand the underlying mechanics and commonalities that these bad performing genes share.

- Acceptance criterea:
    - Enrichr is performed on multiple gene sets of different AUC thresholds (0.5, 0.6, 0.7 etc.).
    - Results of various data sources (kegg, bioplanet, wikipathways etc.) have been obtained in an automated way using the Enrichr API.
    - Results from the Enrichr API have been processed in an understandable plot, an mockup can be found [here](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).


# Enrichr automated API

In [4]:
enrichr_url = 'http://amp.pharm.mssm.edu/Enrichr/'

In [5]:
# Load in initial data
data = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
data.sort_values(by='default_f1', ascending=False)

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
2775,PRKN,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,11,7,4
1659,BUB1B,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,73,66,7
2102,GALNT3,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,12,10,2
1655,PNPLA1,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,0.8,0.2,0.8,31,26,5
2782,TMEM80,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,C10orf2,0.469697,0.0,0.0,1.0,0.0,0.020,0.469697,0.000000,0.0,1.0,0.0,133,132,1
2330,SPTLC2,0.455157,0.0,0.0,1.0,0.0,0.011,0.750374,0.170213,0.0,1.0,0.0,229,223,6
2615,PLCD1,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,8,7,1
2611,UBE2A,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,3,2,1


In [6]:
data.shape[0]

2807

In [7]:
data[['n_tot', 'n_benign', 'n_malign']].sum()

n_tot       303911
n_benign    240104
n_malign     63807
dtype: int64

In [8]:
# Make a dictionary of lists of genes of interest with as key the 0.x threshold

auc_dict = {}
thresholds = np.arange(0.5, 1.01, 0.05).round(decimals=2)
for t in thresholds:
    list_of_genes = data[data['default_auc'] < t]['gene'].tolist()
    auc_dict[t.astype('str')] = '\n'.join(list_of_genes)
    del list_of_genes
del thresholds

# Save it to a file so I don't bombard the Enrichr API with requests
if not os.path.isfile('./user_id_list.json'):
    file = open('./user_id_list.json', 'w')
    file.close()

with open('./user_id_list.json') as json_file:
    try:
        user_list_ids = json.load(json_file)
        json_was_empty = False
    except json.JSONDecodeError:
        user_list_ids = {}
        json_was_empty = True

In [9]:
# Get list ID's for the different AUC's

for threshold in auc_dict.keys():
    if threshold not in user_list_ids.keys():
        description = f'CAPICE performance of genes with an AUC lower than {threshold}'
        payload = {
            'list': (None, auc_dict[threshold]),
            'description': (None, description)
        }
        response = requests.post(enrichr_url + '/addList', files=payload)
        if not response.ok:
            print(f'Something went wrong with {threshold}!')
        else:
            enrichr_addlist_response = json.loads(response.text)
            user_list_ids[threshold] = enrichr_addlist_response['userListId']

# Write it to a file in case I didn't have any.
if json_was_empty:
    with open('./user_id_list.json', 'w') as json_file:
        json.dump(user_list_ids, json_file)

In [10]:
# Now to get the enrichment results.

enrichr_sources_oi = ['BioPlanet_2019', 'WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection',
                     'GO_Biological_Process_2018','Human_Phenotype_Ontology','Jensen_DISEASES',
                     'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET','OMIM_Disease','OMIM_Expanded',
                     'Human_Gene_Atlas', 'ARCHS4_Tissues',
                     'HMDB_Metabolites','Chromosome_Location_hg19']


In [11]:
output_source_link = {}
for source in enrichr_sources_oi:
    output_filename = source + '.csv'
    output_source_link[source] = output_filename

In [12]:
glob_files_present = glob.glob('./EnrichrAPIResults/*.csv')
list_of_files_present = []
if len(glob_files_present) > 0:
    for file in glob_files_present:
        list_of_files_present.append(file.split('/')[-1])

In [13]:
sources_required = []
for source, output_filename in output_source_link.items():
    if output_filename not in list_of_files_present:
        sources_required.append(source)

In [14]:
sources_required

[]

In [15]:
get_enrichr_results(sources_required, user_list_ids, 20)

You should give some sources for me to get data from.


In [16]:
enrichr_data_loc = './EnrichrAPIResults/'

enrichr_dataset = {}

for source in enrichr_sources_oi:
    dataset = enrichr_data_loc + source + '.csv'
    enrichr_dataset[source] = pd.read_csv(dataset)

In [17]:
enrichr_dataset['BioPlanet_2019'].head()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019


In [18]:
for key in enrichr_dataset.keys():
    plot = plot_results(enrichr_dataset[key], 10)
    save(plot)

In [19]:
combined_df = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc][:10], ignore_index=True)
    combined_df = combined_df.append(subdf, ignore_index=True)
combined_df

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
...,...,...,...,...,...,...
1864,1.0,6,chr21,"APP, COL18A1, KCNE1, KCNE2, AIRE, TMPRSS3, ITG...",1.00000,Chromosome_Location_hg19
1865,1.0,7,chr14,"POMT2, ABCD4, SERPINA1, CHD8, PYGL, GPHN, IRF2...",1.00000,Chromosome_Location_hg19
1866,1.0,8,chr22,"SMARCB1, NCF4, MLC1, TCF20, TRIOBP, PEX26, CYP...",1.00000,Chromosome_Location_hg19
1867,1.0,9,chr6 ssto hap7,"SYNGAP1, TNXB, CYP21A2, TUBB, CSNK2B, NEU1, CO...",1.00000,Chromosome_Location_hg19


In [20]:
combined_df.head(20)

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.50,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.50,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.50,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.50,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.50,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
5,0.50,15,Elevation of cytosolic calcium levels,TRPC6,1.00000,BioPlanet_2019
6,0.50,18,Second messenger role in netrin-1 signaling,TRPC6,1.00000,BioPlanet_2019
7,0.50,20,Signaling by activated point mutants of FGFR1,FGF9,1.00000,BioPlanet_2019
8,0.50,21,Caspase-mediated cleavage of cytoskeletal prot...,SPTAN1,1.00000,BioPlanet_2019
9,0.50,22,Tandem pore domain potassium channels,KCNK9,1.00000,BioPlanet_2019


In [21]:
plot = plot_results_in_singleplot_iteractive_legend(combined_df)
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/combined_enrichr_plot_results_interactive_legend.html'

In [22]:
plot = plot_results_in_singleplot(combined_df)
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/combined_enrichr_plot_results.html'

## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


In [23]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
...,...,...,...,...,...,...
3698,1.0,15,chr6 dbb hap3,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3699,1.0,19,chr20,"PTPRT, SNAP25, PANK2, PIGT, MOCS3, ABHD12, JPH...",1.00000,Chromosome_Location_hg19
3700,1.0,17,chr6 qbl hap6,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3701,1.0,20,chr11,"CARS, DPAGT1, CLPB, CTNND1, DNHD1, SBF2-AS1, B...",1.00000,Chromosome_Location_hg19


In [24]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
5,0.5,15,Elevation of cytosolic calcium levels,TRPC6,1.00000,BioPlanet_2019
6,0.5,18,Second messenger role in netrin-1 signaling,TRPC6,1.00000,BioPlanet_2019
7,0.5,20,Signaling by activated point mutants of FGFR1,FGF9,1.00000,BioPlanet_2019
8,0.5,21,Caspase-mediated cleavage of cytoskeletal prot...,SPTAN1,1.00000,BioPlanet_2019
9,0.5,22,Tandem pore domain potassium channels,KCNK9,1.00000,BioPlanet_2019


In [25]:
word_count_df = count_words_in_df(combined_df_ext)

In [26]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

,word,count,auc
0,signaling,14,0.50
1,pathway,14,0.50
2,regulation,12,0.50
3,disease,12,0.50
4,syndrome,9,0.50
587,signaling,16,0.55
588,pathway,13,0.55
589,disease,12,0.55
590,syndrome,11,0.55
591,metabolism,10,0.55


In [27]:
plot = plot_count_results(nwkdf, item='word')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_word_count_bokeh.html'

In [28]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [29]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [30]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [31]:
plot = plot_count_results(ngkdf, item='gene')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_gene_count_bokeh.html'

In [32]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
...,...,...,...,...,...,...
3698,1.0,15,chr6 dbb hap3,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3699,1.0,19,chr20,"PTPRT, SNAP25, PANK2, PIGT, MOCS3, ABHD12, JPH...",1.00000,Chromosome_Location_hg19
3700,1.0,17,chr6 qbl hap6,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3701,1.0,20,chr11,"CARS, DPAGT1, CLPB, CTNND1, DNHD1, SBF2-AS1, B...",1.00000,Chromosome_Location_hg19


In [33]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
5,0.5,15,Elevation of cytosolic calcium levels,TRPC6,1.00000,BioPlanet_2019
6,0.5,18,Second messenger role in netrin-1 signaling,TRPC6,1.00000,BioPlanet_2019
7,0.5,20,Signaling by activated point mutants of FGFR1,FGF9,1.00000,BioPlanet_2019
8,0.5,21,Caspase-mediated cleavage of cytoskeletal prot...,SPTAN1,1.00000,BioPlanet_2019
9,0.5,22,Tandem pore domain potassium channels,KCNK9,1.00000,BioPlanet_2019


In [34]:
word_count_df = count_words_in_df(combined_df_ext)

In [35]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

,word,count,auc
0,signaling,14,0.50
1,pathway,14,0.50
2,regulation,12,0.50
3,disease,12,0.50
4,syndrome,9,0.50
587,signaling,16,0.55
588,pathway,13,0.55
589,disease,12,0.55
590,syndrome,11,0.55
591,metabolism,10,0.55


In [36]:
plot = plot_count_results(nwkdf, item='word')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_word_count_bokeh.html'

In [37]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [38]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [39]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [40]:
plot = plot_count_results(ngkdf, item='gene')
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_gene_count_bokeh.html'

## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


# Correlation analysis Enrichr results
Investigate the interactive legend.

- Bioplanet 2019:
    - Anticorrelation possible, high adjusted-pvalue (APV) when auc is low, but low APV when AUC is high.
    
- Wikipathways:
    - Shows simular anticorrelation to bioplanet, but peaks at 0.55 auc. Could be a normal distribution like.
    
- Kegg 2019:
    - Pretty much all over the place. Some significant when low AUC, but also some not significant. Seems like 2 different distributions are present. One distribution is a linear like from auc 0.5 apv 0.2 to auc 0.7 apv 0.6, but above that a normally distributed curve. Interesting, investigate futher.
    
- Elsevier:
    - Clear normal distribution like, with a peak at either 0.55 (everything is not significant) or 0.65 (largely not significant).

- GO Biological:
    - Similar to kegg, 2 distributions might be shown. A normal distribution like and an inverse exponent. Both seem to be crossing each other at auc 0.5 APV 0.4.

- HPO:
    - Is low and stays low. Tops at auc 0.55 and APV 0.15. Always significant.

- Jensen Diseases:
    - Not really following any distribution. Got some significant, some not and some in between. Tops at auc 0.55 and APV 1.
    
- PheWeb:
    - Exponentially correlated. "Low" when auc is low, get higher when AUC get higher up to auc 0.95 where it becomes more significant.
    
- Clinvar:
    - Multiple distributions present. One 'high' and one 'low'. Investigate futher.
    
- GWAS catalogue:
    - Maybe anti-correlated with some outliers?
    
- DisGeNet:
    - Always significant, simular to HPO.
    
- Omim Disease:
    - Anti-correlated with some points that are always significant.
    
- Omim expanded:
    - Not following omim disease pattern, more like a normal distribution that peaks at auc 0.7 APV 0.6.
    
- Human gene atlas:
    - Like a half cosinus wave with a very sudden drop at auc 0.8.
    
- ARCHS4 tissues:
    - Normal distribution like that tops over APV of 1 at AUC 0.65.
    
- HMDB Metabolites:
    - Says insigificant up untill auc 0.75, where it drops steeply.
    
- Chromosome location:
    - Always insigificant untill AUC 0.85, where chromosome X is always significant.
    
    
# AUC cutoffs where there is still some variation

- Bioplanet: 0.85

- Wikipathways: 0.85

- Kegg: 0.85

- Elsevier pathway collection: 0.80

- GO Biological: 0.80

- HPO: 0.75

- Jensen DISEASES: 0.80

- PheWEB: 0.7 (after that some points become more significant again, but that is commonly seen throughout every database)

- ClinVar: 0.85

- GWAS Catalogue: 0.9

- DisGeNET: 0.7

- OMIM_Disease: 0.85

- OMIM_Expanded: 0.80

- Human Gene Atlas: 0.95

- ARCHS4 tissues: 0.9

- HMDB Metabolites: 0.9

- Chromosome location: Odd one out (always insignificant except for auc 0.9-1.0 where chromosome X becomes significant)


 


In [41]:
for source in combined_df_ext['Source'].unique():
    subset = combined_df_ext[combined_df_ext['Source'] == source]
    subset = subset.groupby(by='AUC').mean()
    rho, pval = stats.spearmanr(subset.index, subset['Adjusted_P-value'])
    print(f'Correlation coefficient for {source} is:{rho}')
    print(f'P-value of {source} is: {pval}')
    if pval >= 0.05:
        print(f'Fail to reject null hypothesis of {source} being uncorrelated, meaning they are uncorrelated.\n')
    else:
        print(f'Able to reject null hypothesis of {source} being uncorrelated, there is a:')
        if rho > 0:
            print('positive correlation. \n')
        else:
            print('negative correlation. \n')

Correlation coefficient for BioPlanet_2019 is:-0.990909090909091
P-value of BioPlanet_2019 is: 3.762571807085399e-09
Able to reject null hypothesis of BioPlanet_2019 being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for WikiPathways_2019_Human is:-0.9363636363636365
P-value of WikiPathways_2019_Human is: 2.208207644917754e-05
Able to reject null hypothesis of WikiPathways_2019_Human being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for KEGG_2019_Human is:-0.881818181818182
P-value of KEGG_2019_Human is: 0.0003301688021839005
Able to reject null hypothesis of KEGG_2019_Human being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for Elsevier_Pathway_Collection is:-0.9363636363636365
P-value of Elsevier_Pathway_Collection is: 2.208207644917754e-05
Able to reject null hypothesis of Elsevier_Pathway_Collection being uncorrelated, there is a:
negative correlation. 

Correlation coefficient for GO_Biological

In [42]:
pheweb = combined_df_ext.where((combined_df_ext['AUC'] <= 0.9) & (combined_df_ext['Source'] == 'PheWeb_2019')).dropna()

In [43]:
pheweb.index = pheweb.reindex(labels=np.arange(pheweb.shape[0]), axis=0).index
grouped_pheweb = pheweb.groupby(by='AUC').mean()
grouped_pheweb
rho, pval = stats.spearmanr(grouped_pheweb.index, grouped_pheweb['Adjusted_P-value'])
print(f'Correlation coefficient for pheweb is:{rho}')
print(f'P-value of pheweb is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of pheweb being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of pheweb being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for pheweb is:0.9153857298880941
P-value of pheweb is: 0.0005337168926636566
Able to reject null hypothesis of pheweb being uncorrelated, there is a:
positive correlation. 



In [44]:
# Both 0.7 and 0.9 state that PheWeb is positively correlated when AUC cutoff is left at 0.9.

In [45]:
kegg = combined_df_ext.where((combined_df_ext['AUC'] <= 0.85) & (combined_df_ext['Source'] == 'KEGG_2019_Human')).dropna()

In [46]:
kegg.index = kegg.reindex(labels=np.arange(kegg.shape[0]), axis=0).index
grouped_kegg = kegg.groupby(by='AUC').mean()
grouped_kegg
rho, pval = stats.spearmanr(grouped_kegg.index, grouped_kegg['Adjusted_P-value'])
print(f'Correlation coefficient for kegg is:{rho}')
print(f'P-value of kegg is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of kegg being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of kegg being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for kegg is:-0.6904761904761906
P-value of kegg is: 0.057990318164572716
Fail to reject null hypothesis of kegg being uncorrelated, meaning they are uncorrelated.



In [47]:
# Further confirms that KEGG might have multiple distributions.

In [48]:
jensen_d = combined_df_ext.where((combined_df_ext['AUC'] <= 0.8) & (combined_df_ext['Source'] == 'Jensen_DISEASES')).dropna()

In [49]:
jensen_d.index = jensen_d.reindex(labels=np.arange(jensen_d.shape[0]), axis=0).index
grouped_jensen_d = jensen_d.groupby(by='AUC').mean()
rho, pval = stats.spearmanr(grouped_jensen_d.index, grouped_jensen_d['Adjusted_P-value'])
print(f'Correlation coefficient for jensen_d is:{rho}')
print(f'P-value of jensen_d is: {pval}')
if pval >= 0.05:
    print(f'Fail to reject null hypothesis of jensen_d being uncorrelated, meaning they are uncorrelated.\n')
else:
    print(f'Able to reject null hypothesis of jensen_d being uncorrelated, there is a:')
    if rho > 0:
        print('positive correlation. \n')
    else:
        print('negative correlation. \n')

Correlation coefficient for jensen_d is:-0.8928571428571429
P-value of jensen_d is: 0.006807187408935392
Able to reject null hypothesis of jensen_d being uncorrelated, there is a:
negative correlation. 



- Look into pathways in the databases themselfs, not the complete database.
    - Final enrichr analysis, investigate best hits, are these significant?
    
        - Is there something here?
        
            -No: No specific enrichment for any pathways.
            
            -Yes: Very good! Investigate further.
            
- Investigate CAPICE performance on UMCG gene panels.

- Train new model on umcg gene panels.
    - 1st option: Top 3 worst and top 3 best by Fingerspitsengefuhl (then put into discussion that more might have been better)
    - 2nd option: Use bad performing genes as panel to train on.
        - Slice out problem genes or genes that share some commonalities?
    - First disease / gene panel specific model training without adding features.
    - Then add new features.



In [50]:
nlines = 0
temp_df = pd.DataFrame(columns=combined_df_ext.columns)
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.05 and rho > 0:
        temp_df = temp_df.append(subset, ignore_index=True)
second_word_count = count_words_in_df(temp_df)
second_word_count.sort_values(by='count', ascending=False)

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond

,word,count,auc
264,"mellitus,",2,0.70
263,diabetes,2,0.70
171,diabetes,2,0.65
172,"mellitus,",2,0.65
79,"mellitus,",2,0.60
...,...,...,...
110,acne,1,0.60
109,externa,1,0.60
108,otitis,1,0.60
107,limbs,1,0.60


In [51]:
combined_df_ext['n_genes'] = combined_df_ext['Overlapping_Genes'].apply(lambda x: split_and_count_words(x))

In [52]:
nlines = 0
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    min_genes = subset['n_genes'].min()
    max_genes = subset['n_genes'].max()
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.05 and rho > 0:
        genes = subset['Overlapping_Genes'].values.tolist()
        output_genes = []
        for gene in genes:
            intermediate_genes = split_and_count_words(gene, return_value = None)
            for g in intermediate_genes:
                if g not in output_genes:
                    output_genes.append(g)
        if len(output_genes) >= 3:
            output_genes = ', '.join(output_genes)
            print(f'Able to reject null hypothesis of {pathway} for AUC: '
                  f'{x.values[0]} - {x.values[-1]} and source: '
                  f'{subset["Source"].unique()[0]} for genes: {output_genes}')
            print('\n')

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond

Able to reject null hypothesis of Transfer RNA aminoacylation for AUC: 0.6 - 0.7 and source: BioPlanet_2019 for genes: CARS, YARS, GARS


Able to reject null hypothesis of Oxytocin signaling pathway for AUC: 0.5 - 0.6 and source: KEGG_2019_Human for genes: KCNJ5, CACNB2, CDKN1A


Able to reject null hypothesis of Tinnitus for AUC: 0.5 - 0.9 and source: PheWeb_2019 for genes: CACNB2, TRPC6, AGBL1, MAP3K7


Able to reject null hypothesis of Acne for AUC: 0.5 - 0.9 and source: PheWeb_2019 for genes: ARHGAP31, NRXN3, KIRREL3


Able to reject null hypothesis of COLONIC MUCOSA for AUC: 0.65 - 0.75 and source: ARCHS4_Tissues for genes: SCN10A, CFHR4, CPA6, NLRP12, PDE11A, SCN11A




In [53]:
nlines = 0
for pathway in combined_df_ext['Term_Name'].unique():
    nlines += 1
    subset = combined_df_ext[combined_df_ext['Term_Name'] == pathway]
    x = subset['AUC']
    y = subset['Adjusted_P-value']
    min_genes = subset['n_genes'].min()
    max_genes = subset['n_genes'].max()
    rho, pval = stats.spearmanr(x, y)
    if pval < 0.05 and rho < 0:
        genes = subset['Overlapping_Genes'].values.tolist()
        output_genes = []
        for gene in genes:
            intermediate_genes = split_and_count_words(gene, return_value = None)
            for g in intermediate_genes:
                if g not in output_genes:
                    output_genes.append(g)
        if len(output_genes) >= 3:
            output_genes = ', '.join(output_genes)
            print(f'Able to reject null hypothesis of {pathway} for AUC: '
                  f'{x.values[0]} - {x.values[-1]} and source: '
                  f'{subset["Source"].unique()[0]} for genes: {output_genes}')
            print('\n')

Able to reject null hypothesis of Netrin-1 signaling for AUC: 0.5 - 0.85 and source: BioPlanet_2019 for genes: TRPC6, HFE2, ROBO1, CDC42, MYO10, RAC1


Able to reject null hypothesis of HDL-mediated lipid transport for AUC: 0.5 - 0.9 and source: BioPlanet_2019 for genes: SCARB1, APOA1, LCAT, APOE, ABCA1, CUBN


Able to reject null hypothesis of Mu-calpain pathway for AUC: 0.5 - 0.85 and source: BioPlanet_2019 for genes: SPTAN1, ACTN1, CAPN1, RAC1


Able to reject null hypothesis of Hypoxia and p53 in the cardiovascular system for AUC: 0.55 - 0.8 and source: BioPlanet_2019 for genes: CDKN1A, UBE2A, TAF1


Able to reject null hypothesis of Ghrelin biosynthesis, secretion, and deacylation for AUC: 0.65 - 0.8 and source: BioPlanet_2019 for genes: PCSK1, IGF1, INS


Able to reject null hypothesis of Retinoblastoma protein regulation for AUC: 0.65 - 0.75 and source: BioPlanet_2019 for genes: CDKN1A, UBTF, TAF1, HDAC3


Able to reject null hypothesis of Bile acid and bile salt metabolism for 

Able to reject null hypothesis of Cholesterol metabolism for AUC: 0.5 - 1.0 and source: KEGG_2019_Human for genes: SCARB1, CYP7A1, PCSK9, APOA1, LCAT, APOE, LRPAP1, LIPG, ABCA1, LDLRAP1, LPA, ABCG8, ABCG5, LRP2, CYP27A1, NPC2, ANGPTL3, CETP, LPL, ABCB11, LIPC, APOB, LDLR, NPC1, APOC2, LIPA, CD36, MYLIP, APOC3, STAR


Able to reject null hypothesis of Phagosome for AUC: 0.5 - 0.75 and source: KEGG_2019_Human for genes: SCARB1, TUBA4A, SFTPA1, ATP6AP1, C3


Able to reject null hypothesis of Prolactin signaling pathway for AUC: 0.6 - 0.8 and source: KEGG_2019_Human for genes: LHCGR, ESR2, JAK2, SOS2, INS


Able to reject null hypothesis of Adrenergic signaling in cardiomyocytes for AUC: 0.8 - 0.9 and source: KEGG_2019_Human for genes: CAMK2B, CACNB2, PLN, AGTR1, PPP2R5D, CACNA1C, CALM1, MYH6, PPP1CB, CAMK2A, ATP2B2, RYR2, TPM3, TPM2, TNNC1, ATP2A2, CACNA1F, ACTC1, PPP2R1A, MYL2, AKT3, CACNA1S, SCN1B


Able to reject null hypothesis of Cushing syndrome for AUC: 0.8 - 0.95 and source: KEGG_

Able to reject null hypothesis of sterol homeostasis (GO:0055092) for AUC: 0.7 - 0.95 and source: GO_Biological_Process_2018 for genes: SCARB1, PCSK9, CYP7A1, GPIHBP1, APOA1, LCAT, APOE, LIPG, ABCA1, LDLRAP1, ABCG8, ABCG5, CAV3, CAV1, NPC2, ANGPTL3, CETP, LPL, APOA5, LIPC, NPC1, APOC2, APOB, LDLR


Able to reject null hypothesis of reverse cholesterol transport (GO:0043691) for AUC: 0.75 - 0.85 and source: GO_Biological_Process_2018 for genes: SCARB1, LCAT, APOA1, APOE, LIPG, ABCA1


Able to reject null hypothesis of cholesterol transport (GO:0030301) for AUC: 0.75 - 0.95 and source: GO_Biological_Process_2018 for genes: SCARB1, LCAT, APOA1, APOE, CEL, LIPG, ABCA1, LDLRAP1, ABCG8, MSR1, ABCG5, CAV1, NPC2, SERAC1, CETP, APOA5, LIPC, NPC1, APOC2, APOB, LDLR


Able to reject null hypothesis of inorganic cation transmembrane transport (GO:0098662) for AUC: 0.75 - 0.85 and source: GO_Biological_Process_2018 for genes: SLC46A1, SCN10A, TRPC6, ATP6AP1, SCN11A, SCN3A, TRPM4, HCN1, SCN9A, SLC4A

Able to reject null hypothesis of Autosomal dominant inheritance (HP:0000006) for AUC: 0.5 - 1.0 and source: Human_Phenotype_Ontology for genes: YARS, RTN2, KCNJ5, NTRK2, CEACAM16, KCNK9, MAPK8IP3, GIGYF2, KBTBD13, CACNB2, LMBR1, ARHGAP31, SPTLC2, FGF9, GJB3, MYH8, C10ORF2, NSMF, ANG, SPTAN1, TRPM4, KCNE3, KRT86, NLRP1, CPA6, PDYN, CYP2A6, PLCD1, LHCGR, SCN10A, GARS, DFNA5, KCNE2, TGM6, KLF11, SCO2, AAGAB, MATN3, RPL5, PCSK9, PUF60, CHRNB2, ACTN1, ROM1, KIAA0196, FAM111B, TGFBI, JPH2, SCN11A, JAK2, CDON, APOA1, CEL, MAF, COL4A3, FOXC2, NFIX, CRYBA1, PPM1D, PTHLH, PDE11A, SEPN1, ZBTB18, MAP3K1, CFC1, RAX2, NLRP12, PNKD, AGBL1, TARDBP, GUCA1A, ACTG2, FGF8, SCN9A, SOX10, OPTN, EPHA2, HEPACAM, TERC, CHRNA4, CACNA1C, MYH14, OTX2, SMAD6, CHMP2B, MKRN3, CALM1, THRA, NOG, AMPD1, FOXL2, KIF2A, MYH6, RPL11, INS, PLN, GNA11, TBX6, MIB1, RAD51, PRSS1, RBPJ, ACTG1, SOX2, RPS19, PIEZO1, KIF21A, IFITM5, PITX3, KIRREL3, G6PD, AIP, ANK2, BSCL2, COL4A2, LPA, CUL3, PSEN1, NEUROD1, PTDSS1, ANKH, ZNF423, A



Able to reject null hypothesis of Kidney cancer for AUC: 0.6 - 1.0 and source: Jensen_DISEASES for genes: CEP104, SCARB1, LHCGR, GRID2, UNC13A, TRPC6, KCNK9, NRXN3, PDYN, AFF4, GIGYF2, ROBO1, CACNB2, SCN10A, NLRP7, MYH8, DCHS1, NLRP1, ALPK1, SPTAN1, PLCD1, TLR3, CASC5, IDH1, TAF1, PCSK9, NLRC4, ARID2, ACTN1, ARHGAP29, SPECC1L, KCNQ5, NAA15, SAMD9L, ABCA13, KIAA1109, SCN11A, JAK2, CDON, EBF3, PHIP, SOS2, SAMD9, C3, PDE11A, SCN3A, MYO1E, NLRP12, HCN1, DOCK3, TUSC3, SIN3A, SCN9A, SRRM2, NSUN2, DNMT3A, RBP3, MYH6, CACNA1C, MYH14, XIRP2, BPTF, GRIA4, BRPF1, BCL11A, CLCN4, SPAG1, TRIP12, POGZ, KIF21A, SBF1, ANK2, TBL1XR1, LPA, CACNA1H, PTDSS1, ADAMTS17, ZNF423, C5ORF42, GRIA3, ABCA1, SMAD2, ANKRD26, NBEAL2, A2ML1, EXT1, CDK13, SETD5, LAMC3, CHD8, CLTC, DOCK7, CTCF, PYGL, FMN2, BICD2, VWF, MYH2, WAC, LRP6, GANAB, CUX2, MYO10, ATP2B2, TSHR, REST, TRIO, MYT1L, CTNND2, ZNF292, PLOD2, SMCHD1, TBK1, GNPTAB, PPP2R1A, EP300, MAP3K7, KCNH1, RAG1, CUBN, MAP2K1, PHKA1, ACSL4, KRT9, AR, KAT6B, KAT6A, 

Able to reject null hypothesis of brugada syndrome for AUC: 0.5 - 0.9 and source: ClinVar_2019 for genes: CACNB2, KCNE3, CACNA1C, SCN1B


Able to reject null hypothesis of amyotrophic lateral sclerosis for AUC: 0.5 - 0.85 and source: ClinVar_2019 for genes: ANG, NEFH, TARDBP, OPTN


Able to reject null hypothesis of autosomal dominant non-syndromic sensorineural deafness type dfna for AUC: 0.5 - 1.0 and source: ClinVar_2019 for genes: GJB3, DIAPH1, MYH14, COCH, ACTG1, GRHL2, POU4F3, GJB2, WFS1, COL11A2, KCNQ4, MYO7A, TMC1, EYA4, P2RX2, MYO6, MYH9, TECTA


Able to reject null hypothesis of distal arthrogryposis for AUC: 0.5 - 0.9 and source: ClinVar_2019 for genes: MYH8, MYBPC1, MYH3, TPM2


Able to reject null hypothesis of hereditary hearing loss and deafness for AUC: 0.5 - 1.0 and source: ClinVar_2019 for genes: GJB3, TWNK, CLDN14, COL4A3, PDZD7, SMPX, MYH14, SOX10, POU3F4, DCDC2, ATP2B2, COCH, ACTG1, DIAPH1, GIPC3, SDHAF1, GPSM2, PRPS1, PTPRQ, DGUOK, PDHX, MGME1, EDN3, EYA1, ESRRB, 

Able to reject null hypothesis of Pediatric failure to thrive for AUC: 0.55 - 1.0 and source: DisGeNET for genes: CD96, ARHGAP31, SLC25A3, SLC46A1, TGDS, NDUFA1, HNRNPH2, SNX10, NLRP1, IGF1, BAAT, ROBO1, MUT, GPIHBP1, RPL5, SLC10A2, VPS33B, DOLK, NLRC4, MPDU1, NFIX, GMNN, NACC1, CDON, SAR1B, INS, ORC1, TALDO1, TBR1, SUGCT, NDUFAF4, MKRN3, SETD5, PRSS1, RBPJ, ACTG1, PPP1CB, NIPAL4, RPS19, KDM6A, TMEM216, PKP1, GBA, HOXD13, DLL4, HSD11B2, PTDSS1, SLC25A24, NR0B1, EXT1, CYP11B2, NDUFA12, SMC3, ACTB, GJA1, GNPTAB, EP300, SOX9, MAP3K7, RAG1, MAP2K1, TPM3, TPM2, TINF2, ARMC5, KAT6B, ETHE1, KMT2D, GCDH, KMT2A, GATA6, DGUOK, RAD21, RBM10, FUCA1, TMPRSS15, EXT2, SON, GNB1, IARS, LHX4, ADA, RERE, ITGB4, NIPBL, SERAC1, JAK3, ACADS, TTC37, AVPR2, HBA2, MYH3, PEX3, PEX6, CLDN16, NDUFB9, GALT, HDAC8, EPCAM, BCAP31, ASXL1, ASXL3, PLP1, MBTPS2, OTC, NEUROG3, NEXMIF, PRF1, ABCA12, NFU1, PNP, PIEZO2, MPV17, EARS2, SLC34A3, ALG9, MCCC1, ALG3, MOGS, BCS1L, RTEL1, SLC22A5, AARS2, PEX11B, CLCNKB, XPC, CYP27

Able to reject null hypothesis of Mental Retardation for AUC: 0.75 - 1.0 and source: DisGeNET for genes: CEP104, SLC46A1, CTDP1, NFIX, RBM8A, KCNK9, PRCD, PPM1D, AFF4, PORCN, COL4A3BP, DPP6, NACC1, HIST1H1E, APOE, ARID2, SCN3A, ZBTB18, TRAPPC2, CD96, BRF1, ROM1, TWNK, NAGA, EBF3, IGF1, UBE2A, MUT, ARHGAP31, MAF, SPECC1L, DCHS1, HNRNPH2, ZMYND11, HCN1, DDX3X, THRA, TUSC3, PHF6, IFIH1, STS, SIN3A, ANKRD1, SRRM2, NSUN2, DNMT3A, SOX11, TBR1, PPP2R5D, SOX10, RBP3, HEPACAM, SAR1B, TMEM240, ORC1, GRIA4, BRPF1, NAA10, BCL11A, TMCO1, RAD51, CLCN4, ASXL2, AMHR2, HNRNPU, ACTG1, POGZ, KIRREL3, G6PD, BSCL2, TBL1XR1, ANOS1, ARL6, GATA1, PTDSS1, RDH12, PGK1, ZNF423, GRIA3, ABCA1, SYT1, NONO, NR2F1, AMMECR1, NR0B1, EXT1, CCDC88C, CDK13, SETD5, CTCF, FMN2, CHMP1A, RAC1, KDM6A, PDE4D, PAX6, TMEM216, DCX, PIGL, CDC42, TTR, USP9X, TSHR, PGAP3, MYT1L, SMC3, ACTB, GJA1, PPP2R1A, AKT3, PDHX, SLC6A19, TINF2, ACSL4, HNF1A, CYP27A1, AR, ETHE1, ZNF711, CERKL, SRCAP, FOXG1, GATA6, NEU1, CTSA, BCL11B, FUCA1, GLUD1

Able to reject null hypothesis of Leukemia promyelocytic-HL-60 for AUC: 0.8 - 0.9 and source: Human_Gene_Atlas for genes: NDUFAF4, BPTF, CR2, CDKN2A, POLR1C, SMC3


Able to reject null hypothesis of OlfactoryBulb for AUC: 0.9 - 1.0 and source: Human_Gene_Atlas for genes: INF2, SGCB, MPZ, PRX, SOX10, CLCN2, BMP4, EVC, PMP22, CYP1B1, LPL, DMD, CRB2, ARHGEF10, VPS13B, PRICKLE1, NDRG1, SGCD, ERBB3, NEK1, FREM1


Able to reject null hypothesis of SUPERIOR FRONTAL GYRUS for AUC: 0.5 - 0.9 and source: ARCHS4_Tissues for genes: RTN2, NTRK2, DPP6, UNC13A, KCNK9, FGF9, NRXN3, NSMF, MAPK8IP3, CHRNB2, PCSK1, PRCD, ROM1, PDYN, KRT86, CHRDL1, CYP26B1, KCNQ5, NEFH, GPIHBP1, CAMK2B, NFIX, TREM2, PTHLH, MC4R, PORCN, NACC1, APOE, SCN3A, P2RY12, HCN1, DOCK3, THRA, GUCA1A, RS1, TBR1, SOX10, POU3F3, POU3F4, PKDCC, MYRF, CHRNA4, CACNA1C, TMEM240, THSD1, SOHLH1, MYH14, GRIA4, CLCN4, FMN2, GLDN, SOX2, KIRREL3, MYBPC1, VWF, ANK2, BSCL2, TUBB2A, VAMP1, LPA, ATL1, CAMK2A, CACNA1H, CACNA1G, DLL4, CUX2, RDH12, ZNF

Able to reject null hypothesis of BRAIN (BULK) for AUC: 0.5 - 0.9 and source: ARCHS4_Tissues for genes: RTN2, NTRK2, DPP6, UNC13A, KCNK9, FGF9, NRXN3, NSMF, MAPK8IP3, GRID2, PRCD, PDYN, SLC6A9, PCSK1, CHRDL1, NEFH, GPIHBP1, CHRNB2, ROM1, CYP26B1, KCNQ5, CAMK2B, NFIX, PTHLH, MC4R, APOE, SCN3A, P2RY12, ZBTB18, HCN1, DOCK3, SMPX, MYRF, THRA, CHRNA4, CACNA1C, TMEM240, RS1, STS, SOHLH1, MYH14, GRIA4, TBR1, SOX10, POU3F3, CLCN4, FMN2, GLDN, SOX2, KIRREL3, MYBPC1, VWF, PAX6, ANK2, BSCL2, TUBB2A, MFRP, VAMP1, ATL1, CAMK2A, CACNA1G, CUX2, RDH12, ZNF423, GRIA3, PRNP, DTNA, SYT1, NR2F1, ATP2B2, MYT1L, CTNND2, GJA1, SCN1B, KCNH1, STX1B, EEF1A2, COL4A5, CTTNBP2, FOXG1, DVL1, BDNF, GJB1, NBEA, DLG3, DLG4, PAH, COL9A3, COL9A2, RYR2, KCNC1, ITGB4, AFF2, KCNT1, CTSF, PLEKHG5, HBA2, DNM1, TUBB2B, KCNMA1, IL1RAPL1, CRYAB, MOCS1, SHANK3, KCNA2, LRP2, PLP1


Able to reject null hypothesis of BREAST (BULK TISSUE) for AUC: 0.5 - 1.0 and source: ARCHS4_Tissues for genes: NTRK2, SCARB1, ARHGAP31, GJB3, GPD1, D

In [54]:
combined_df_ext[combined_df_ext['Term_Name'] == 'Tinnitus']

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source,n_genes
1540,0.50,4,Tinnitus,"CACNB2, TRPC6",0.218952,PheWeb_2019,2
1561,0.55,6,Tinnitus,"CACNB2, TRPC6",0.630421,PheWeb_2019,2
1582,0.60,6,Tinnitus,"CACNB2, TRPC6",0.774783,PheWeb_2019,2
1601,0.65,7,Tinnitus,"CACNB2, TRPC6",0.901722,PheWeb_2019,2
1620,0.70,10,Tinnitus,"CACNB2, TRPC6",0.968591,PheWeb_2019,2
1640,0.75,1,Tinnitus,"CACNB2, TRPC6, AGBL1",1.000000,PheWeb_2019,3
1662,0.80,2,Tinnitus,"CACNB2, TRPC6, AGBL1",1.000000,PheWeb_2019,3
1686,0.85,6,Tinnitus,"CACNB2, TRPC6, AGBL1",1.000000,PheWeb_2019,3
1702,0.90,4,Tinnitus,"CACNB2, TRPC6, AGBL1, MAP3K7",1.000000,PheWeb_2019,4


In [55]:
combined_df_ext[combined_df_ext['Term_Name'].str.lower().str.startswith('brugada')]

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source,n_genes
1324,0.50,3,Brugada syndrome,"CACNB2, TRPM4",0.281150,Jensen_DISEASES,2
1341,0.55,2,Brugada syndrome,"CACNB2, KCNE3, TRPM4",0.034910,Jensen_DISEASES,3
1362,0.60,3,Brugada syndrome,"CACNB2, KCNE3, TRPM4",0.031890,Jensen_DISEASES,3
1383,0.65,4,Brugada syndrome,"CACNB2, KCNE3, TRPM4",0.038183,Jensen_DISEASES,3
1406,0.70,7,Brugada syndrome,"CACNB2, KCNE3, TRPM4",0.042063,Jensen_DISEASES,3
1429,0.75,10,Brugada syndrome,"CACNB2, KCNE3, TRPM4",0.086863,Jensen_DISEASES,3
1449,0.80,11,Brugada syndrome,"CACNB2, KCNE3, CACNA1C, TRPM4",0.011256,Jensen_DISEASES,4
1762,0.50,5,brugada syndrome,CACNB2,0.621492,ClinVar_2019,1
1773,0.55,1,brugada syndrome,"CACNB2, KCNE3",0.100268,ClinVar_2019,2
1793,0.60,1,brugada syndrome,"CACNB2, KCNE3",0.123679,ClinVar_2019,2


- Genes involved in identifying brugada syndrome might be troublesome for CAPICE, but then again, if we look at the command 'combined_df_ext[combined_df_ext['Term_Name'].str.lower().str.startswith('brugada')]' there's not a lot of genes in that.
- Colonic Mucosa is not really that interesting either. Starts at AUC 0.65 and ends at 0.75, only increasing slightly with an adjusted p-value of at least 0.999925 (very insignificant).
    - Transfer RNA aminoacylation is a bit different, still insignificant but starts at 0.594633.
    - Same goes for Oxytocin signaling pathway, starts at 0.941531
    - Tinnitus is a bit different, only has 2 genes in the lower AUC values but starts out at 0.218952 and goes up to 1 from auc 0.5-0.9.
    
    
- When a pad performing gene panels is found, make a decision tree on what to do next.
- Keep in mind, I have 4 times the amount of benign samples than malignant samples.
- Chi squared / regression correlation analysis between false positive / negative samples on a shared feature.
- Turning in concept verion paper on 6th of July is possible with the 2 weeks deadline for feedback. 
- All methods to seperate python files and then import them to clean up notebook.
- Phase 2 will produce graphs, order them to the story I want to tell.
    - Thesis is a bit more than a paper, but a bit less than a bachelor

In [100]:
umcg_genepanels = pd.read_excel('/home/rjsietsma/Documents/LAB-F0701 Excel van Genenlijst panels Genoomdiagnostiek UMCG_versie 4 mei.xlsx', sheet_name=None)

In [152]:
start = '5GPM'
output_genepanels = {}
in_genepanels = False
for value in umcg_genepanels.values():
    subject = value.columns[0]
    if subject == start:
        in_genepanels = True
    if in_genepanels:
        specific_disease = value.iloc[0,0]
        if isinstance(specific_disease, float):
            specific_disease = subject
        first_col = subject
        real_col = 'Gen'
        gen_index = value[value[first_col] == real_col].index
        if gen_index.shape[0] < 1:
            continue
        if value.shape[1] > 1:
            value = value.iloc[:,0]
        value = value.shift(-(gen_index.values[0] + 1))
        if not isinstance(value, pd.DataFrame):
            value = pd.DataFrame(value)
        value.columns = [real_col]
        value.replace('Terug naar inhoudsopgave', np.NaN, inplace=True)     
        value.dropna(inplace=True)
        output = value.values.tolist()
        real_output = []
        for i in output:
            real_output = real_output + i
        if subject not in output_genepanels.keys():
            temp = {specific_disease: real_output}
            output_genepanels[subject] = temp
        else:
            temp = {specific_disease: real_output}
            subject_dict = output_genepanels[subject]
            subject_dict.update(temp)

output_genepanels

{'5GPM': {'5GPM': ['A2M',
   'A2ML1',
   'A4GALT',
   'AAAS',
   'AAGAB',
   'AARS',
   'AARS2',
   'AASS',
   'ABAT',
   'ABCA1',
   'ABCA12',
   'ABCA3',
   'ABCA4',
   'ABCA5',
   'ABCA7',
   'ABCB1',
   'ABCB11',
   'ABCB4',
   'ABCB6',
   'ABCB7',
   'ABCC11',
   'ABCC2',
   'ABCC6',
   'ABCC8',
   'ABCC9',
   'ABCD1',
   'ABCD2',
   'ABCD3',
   'ABCD4',
   'ABCG2',
   'ABCG5',
   'ABCG8',
   'ABHD12',
   'ABHD5',
   'ABL1',
   'ABO',
   'ACAD8',
   'ACAD9',
   'ACADL',
   'ACADM',
   'ACADS',
   'ACADSB',
   'ACADVL',
   'ACAN',
   'ACAT1',
   'ACBD5',
   'ACE',
   'ACHE',
   'ACKR1',
   'ACO2',
   'ACOX1',
   'ACOX2',
   'ACP4',
   'ACP5',
   'ACSF3',
   'ACSL4',
   'ACSL6',
   'ACTA1',
   'ACTA2',
   'ACTB',
   'ACTC1',
   'ACTG1',
   'ACTG2',
   'ACTL6A',
   'ACTL6B',
   'ACTN1',
   'ACTN2',
   'ACTN3',
   'ACTN4',
   'ACVR1',
   'ACVR1B',
   'ACVR2B',
   'ACVRL1',
   'ACY1',
   'ADA',
   'ADA2',
   'ADAM10',
   'ADAM17',
   'ADAM22',
   'ADAM9',
   'ADAMTS10',
   'ADAMTS13',


In [166]:
auc_data = data[['gene','default_auc']].copy()
auc_data.columns = ['gene','auc']

In [173]:
auc_data.set_index('gene', inplace=True)

In [179]:
auc_data

,auc
gene,
RDH12,0.839161
EIF2AK4,0.981481
EIF2AK3,0.966667
PRKAG2,0.981203
JAG1,0.925388
...,...
FLG2,1.000000
ACOX2,1.000000
SLITRK4,1.000000


In [177]:
x = auc_data.index.values
x2 = auc_data['auc'].values
auc_data_series = pd.Series(x2, index=x)

In [181]:
auc_data_dict = auc_data_series.to_dict()

In [195]:
worst_performer = None
worst_performer_auc = 1
worst_performer_std = None
worst_performer_min = None
worst_performer_max = None
worst_performing_n = 0
worst_performing_ntot = 0
for key in output_genepanels.keys():
    for key2, value in output_genepanels[key].items():
        print(f'Disease: {", ".join([key, key2])}')
        n_genes_tot = len(value)
        keep_track = np.full(shape=len(value), fill_value=np.NaN)
        for iteration, gene in enumerate(value):
            if gene in auc_data_dict.keys():
                keep_track[iteration] = auc_data_dict[gene]
        keep_track = keep_track[~np.isnan(keep_track)]
        average_auc = keep_track.mean()
        median_auc = np.median(keep_track)
        n_used = keep_track.size
        p_used = n_used / n_genes_tot * 100
        print(f'Total used genes: {round(p_used)}% (n: {n_used}/{n_genes_tot})')
        print(f'Minimal AUC: {keep_track.min()}')
        print(f'Maximum AUC: {keep_track.max()}')
        print(f'Average AUC: {average_auc} (+-{keep_track.std()})')
        print(f'Median AUC: {median_auc}\n')
        if average_auc < worst_performer_auc:
            worst_performer = ', '.join([key, key2])
            worst_performer_auc = average_auc
            worst_performer_std = keep_track.std()
            worst_performer_min = keep_track.min()
            worst_performer_max = keep_track.max()
            worst_performing_n = n_used
            worst_performing_ntot = n_genes_tot

print(f'The worst performing UMCG gene panel is: {worst_performer}')
print(f'With an average AUC of: {worst_performer_auc} (+-{worst_performer_std})'
      f'(n: {worst_performing_n}/{worst_performing_ntot})')
print(f'And a minimum AUC of: {worst_performer_min} and maximum of: {worst_performer_max}')
        
            
            

Disease: 5GPM, 5GPM
Total used genes: 59% (n: 2603/4385)
Minimal AUC: 0.0
Maximum AUC: 1.0
Average AUC: 0.9144860084578842 (+-0.1079915059830671)
Median AUC: 0.9469696969696968

Disease: Neurogenetica, Dementie
Total used genes: 100% (n: 18/18)
Minimal AUC: 0.7291666666666667
Maximum AUC: 1.0
Average AUC: 0.8912414633016061 (+-0.0866484280970039)
Median AUC: 0.9293471516655678

Disease: Neurogenetica, Parkinson/parkinsonisme
Total used genes: 77% (n: 17/22)
Minimal AUC: 0.482484076433121
Maximum AUC: 1.0
Average AUC: 0.9065114963007039 (+-0.13925503888226975)
Median AUC: 0.95

Disease: Neurogenetica, Neurodegeneratie met ijzerafzetting in de hersenen (NBIA)
Total used genes: 100% (n: 9/9)
Minimal AUC: 0.9099264705882352
Maximum AUC: 1.0
Average AUC: 0.9645365734629314 (+-0.031676345861338405)
Median AUC: 0.962962962962963

Disease: Neurogenetica, Dandy Walker malformatie
Total used genes: 71% (n: 5/7)
Minimal AUC: 0.9
Maximum AUC: 1.0
Average AUC: 0.9497222222222224 (+-0.04282104593898